In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

# --- CONFIGURATION ---
WARS_TO_PLAY = 5

# Payoff Matrix
PAYOFFS = {
    ('Restrain', 'Restrain'): (3, 3),  # Peace
    ('Restrain', 'Attack'):   (0, 5),  # Sucker
    ('Attack', 'Restrain'):   (5, 0),  # Temptation
    ('Attack', 'Attack'):     (1, 1),  # War
}

# --- STRATEGIES ---
def get_strategy_move(strat_name, my_history, opp_history):
    if strat_name == "Tit-for-Tat":
        return opp_history[-1] if opp_history else 'Restrain'

    elif strat_name == "Grim Trigger":
        if 'Attack' in opp_history: return 'Attack'
        return 'Restrain'

    elif strat_name == "Tit-for-Two-Tats":
        # Only attack if opponent attacked TWICE in a row
        if len(opp_history) >= 2 and opp_history[-1] == 'Attack' and opp_history[-2] == 'Attack':
            return 'Attack'
        return 'Restrain'

    elif strat_name == "Always Defect":
        return 'Attack'

    elif strat_name == "Mad Man":
        # Randomly choose 50/50
        return random.choice(['Restrain', 'Attack'])

    return 'Restrain' # Default

# --- SIMULATION ENGINE ---
def run_war(war_id, total_days, user_role, user_strat):
    # Calculate the day the betrayal happens
    mid_day = (total_days // 2) + 1

    my_hist, opp_hist = [], []
    my_score, opp_score = 0, 0

    # Calculate Max Possible Score (Perfect Peace)
    max_peace_score = total_days * 3

    print(f"\n⚔️ WAR {war_id} STARTING (Days: {total_days})")
    print(f"   You are: {user_role}")
    print(f"   Strategy: {user_strat}")
    print(f"   BENCHMARK: Highest Possible Peace Score is {max_peace_score}")
    print(f"   WARNING: Opponent will break truce on Day {mid_day}!")

    for day in range(1, total_days + 1):
        # 1. ENEMY MOVE (Default: Tit-for-Tat)
        if day == 1:
            opp_move = 'Restrain'
        else:
            opp_move = my_hist[-1] # They copy you

        # 2. YOUR MOVE
        user_move = get_strategy_move(user_strat, my_hist, opp_hist)

        # 3. THE BETRAYAL EVENT
        # Regardless of if you are Player 1 or Player 2, the OPPONENT attacks.
        note = ""
        if day == mid_day:
            opp_move = 'Attack'
            note = "⚠️ [OPPONENT BETRAYAL]"

        # 4. SCORING
        p_me, p_opp = PAYOFFS[(user_move, opp_move)]
        my_score += p_me
        opp_score += p_opp

        my_hist.append(user_move)
        opp_hist.append(opp_move)

        # Optional: Print betrayal for visibility
        if day == mid_day:
             print(f"   Day {day}: You {user_move} vs Enemy {opp_move} -> {note}")

    print(f"   🏁 RESULT: You {my_score} - {opp_score} Enemy")
    return my_score, opp_score, max_peace_score

# --- MAIN INTERFACE ---

def main():
    print("--- REPEATED GAMES: SURVIVAL MODE ---")

    try:
        duration = int(input("Enter number of days for each war: "))
    except:
        duration = 20 # Default

    results = {
        "War": [],
        "User Score": [],
        "Enemy Score": [],
        "Strategy": [],
        "Benchmark": []
    }

    # Loop for 5 Wars
    for i in range(1, WARS_TO_PLAY + 1):
        print(f"\n--- SETUP FOR WAR {i} OF {WARS_TO_PLAY} ---")

        # Select Role
        print("Choose Role:")
        print("1. Player 1")
        print("2. Player 2")
        r_choice = input("Choice (1/2): ")
        role = "Player 1" if r_choice == "1" else "Player 2"

        # Select Strategy
        print("Choose Your Strategy:")
        print("1. Tit-for-Tat (Standard)")
        print("2. Grim Trigger (Unforgiving)")
        print("3. Tit-for-Two-Tats (Forgiving)")
        print("4. Always Defect (Hawk)")
        print("5. Mad Man (Random)")
        s_choice = input("Choice (1-5): ")

        strat_map = {
            "1":"Tit-for-Tat",
            "2":"Grim Trigger",
            "3":"Tit-for-Two-Tats",
            "4":"Always Defect",
            "5":"Mad Man"
        }
        strategy = strat_map.get(s_choice, "Tit-for-Tat")

        # Run Sim
        u_score, e_score, max_possible = run_war(i, duration, role, strategy)

        # Store Data
        results["War"].append(f"War {i}\n({strategy})")
        results["User Score"].append(u_score)
        results["Enemy Score"].append(e_score)
        results["Strategy"].append(strategy)
        results["Benchmark"].append(max_possible)

    # --- VISUALIZATION ---
    print("\ngenerating graphs...")

    df = pd.DataFrame(results)

    # Plotting
    x = np.arange(len(df["War"]))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, df["User Score"], width, label='You', color='royalblue')
    rects2 = ax.bar(x + width/2, df["Enemy Score"], width, label='Enemy (Traitor)', color='firebrick')

    # Add Benchmark Line (Total Peace)
    # We take the first value since duration is constant
    benchmark_score = df["Benchmark"][0]
    ax.axhline(y=benchmark_score, color='green', linestyle='--', linewidth=2, label=f'Perfect Peace ({benchmark_score} pts)')

    ax.set_ylabel('Total Points Scored')
    ax.set_title(f'War Simulation Results ({duration} Days)')
    ax.set_xticks(x)
    ax.set_xticklabels(df["War"])
    ax.legend()

    # Add labels on top of bars
    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)

    plt.tight_layout()
    plt.show()

    # Display Data Table
    print("\n--- FINAL STATISTICS ---")
    print(df[["War", "Strategy", "User Score", "Enemy Score", "Benchmark"]])

if __name__ == "__main__":
    main()